In [1]:
import pandas

from GA import GA
import optproblems.cec2005 as optproblems

# Test Functions
- F2: Shifted Schwefel’s Problem 1.2
    - Uni-modal without noise
- F4: Shifted Schwefel’s Problem 1.2 with Noise in Fitness
    - Uni-modal with noise
- F8: Shifted Rotated Ackley’s Function with Global Optimum on Bounds
    - Single function
    - Global optimum on bounds
    - Multi-modal
- F13: Shifted Expanded Griewank’s plus Rosenbrock’s Function (F8F2)
    - Expanded Function
    - Multi-modal
- F17: Rotated Version of Hybrid Composition Function with Noise in Fitness
    - Gaussian noise in fitness
    - Multi-modal
    - Huge number of local optima
    - In-deterministic
- F24: Rotated Hybrid Composition Function
    - Composite function
    - Huge number of local optima
    - Multi-modal

Hyper parameters

In [4]:
# F2 :   Unimodal without noise
# F4 :   Unimodal with noise
# F8 :   Single Function, Global Optimum on Bounds, Multi-modal
# F13:   Expanded Function, Multi-modal
# F17:   Gaussian noise in Fitness, Multi-modal, Huge number of local optima, indeterministic
# F24:   Composite Function, Huge number of local optima, Multi-modal

# benchmarks_and_bounds = [optproblems.F2, [-100,100], optproblems.F4, [-100,100], optproblems.F8, [-32,32], optproblems.F13, [-3,1], optproblems.F17, [-5,5], optproblems.F24, [-5,5]]
# no_dimensions = [10, 30, 50]
# pop_sizes = [5, 10, 12, 15]
# num_generations = [10, 50, 100, 150, 1000, 2000]
# tournament_t = [2, 3, 4]
# crossover_percentage = [0.5, 0.95, 1]
# mutation_rates = [0.2, 0.8, 1]
# decreasing_mutation_rate = [True, False]
# elites = [0, 1]

benchmarks_and_bounds = [optproblems.F2, [-100,100], optproblems.F4, [-100,100]]
no_dimensions = [10, 30]
pop_sizes = [5, 10]
num_generations = [10, 50]
tournament_t = [2, 3]
crossover_percentage = [0.5]
mutation_rates = [0.2, 1]
decreasing_mutation_rate = [True, False]
elites = [0, 1]

average_range = 5

result = []
for i in range(0, len(benchmarks_and_bounds), 2):
    bound = benchmarks_and_bounds[i+1]
    for no_dimensions in no_dimensions:
        benchmark = benchmarks_and_bounds[i](no_dimensions)
        optimal_solution = benchmark.get_optimal_solutions()
        benchmark.evaluate(optimal_solution[0])
        optimal_minimum = optimal_solution[0].objective_values
        for pop_size in pop_sizes:
            for t in tournament_t:
                for crossover in crossover_percentage:
                    for mutation in mutation_rates:
                        for dmr in decreasing_mutation_rate:
                            for elite in elites:
                                for generations in num_generations:
                                    best_fitness = 0
                                    average_fitness = 0
                                    best_time = 0
                                    average_difference_from_optimal = 0
                                    best_difference_from_optimal = 0
                                    for j in average_range:
                                        algorithm = GA(benchmark, no_dimensions, bound, pop_size, num_generations=generations, t=t, crossover_percentage=crossover, mutation_rate=mutation, decreasing_mutation_rate=dmr, elite=elite)
                                        result, time = algorithm.run()
                                        fitness = result.fitness
                                        diff_from_optimal = fitness - optimal_minimum

                                        average_fitness += 1/average_range * fitness
                                        average_difference_from_optimal += 1/average_range * diff_from_optimal

                                        if j == 0:
                                            best_fitness = fitness
                                            best_difference_from_optimal = diff_from_optimal
                                            best_time = time
                                            continue

                                        if fitness < best_fitness:
                                            best_fitness = fitness

                                        if diff_from_optimal < best_difference_from_optimal:
                                            best_difference_from_optimal = diff_from_optimal




df = pandas.DataFrame(result)
df.to_pickle("df.pkl")
# bound = (-100, 100)
# pop_size = 3
# benchmarks = {optproblems.cec2005.F14(NO_dimensions)}
# opt = benchmark.get_optimal_solutions()
#
# algorithm1 = GA(benchmark, NO_dimensions, bound, pop_size, num_generations=100, elite=1)
# algorithm1.run()

KeyboardInterrupt: 